In [ ]:
import os
import re
import pandas as pd
pd.options.display.max_colwidth = 20000
import nltk
import sklearn.feature_extraction.text
import spacy

In [ ]:
pathName = r'C:\Users\agupt\Documents\Courses\MGTF 423'
fileName = 'McKesson10K2022.txt'
cleanFile = open(pathName +'\\'+ 'clean_' + fileName,'r')
content = cleanFile.read()

In [ ]:
# A DataFrame for Text Data

lines = content.splitlines()
i = 0
j = 0
d = []
for l in lines:
    sent = re.split(r'\. |:',l)
    for s in sent:       
        d.append([i,j,s.strip()])
        j = j+1
    i = i+1
    j = 0
ldf = pd.DataFrame(d, columns = ['Line', 'Sent', 'Content'])
ldf

***STATISTICAL OPERATIONS***

In [ ]:
def compute_freq(text, n_value=2):

    tokens = nltk.word_tokenize(text)
    ngrams = nltk.ngrams(tokens, n_value)
    ngram_fdist = nltk.FreqDist(ngrams)
    return ngram_fdist

In [ ]:
text = content
freq_dist = compute_freq(text)
topk = freq_dist.most_common(50)
print(topk)

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

In [ ]:
def compute_freq_minus_stopwords(text, n_value=2):
    from nltk.tokenize import WhitespaceTokenizer
    Tokenizer=WhitespaceTokenizer()
    text = re.sub(r'[^\w\s]', '', text) # replace any character except alphanumeric and whitespace with null
    english_stopwords = stopwords.words('english')
    tokens = Tokenizer.tokenize(text)
    filtered_tokens = [t for t in tokens if t not in english_stopwords]
    ngrams = nltk.ngrams(filtered_tokens, n_value)
    ngram_fdist = nltk.FreqDist(ngrams)
    return ngram_fdist

In [ ]:
text = re.sub(r'[^\w\s]', '', 'ab,c d"e!f')
text

In [ ]:
text = content
freq_dist = compute_freq_minus_stopwords(text,3)
topk = freq_dist.most_common(100)
topk

In [ ]:
freq_table = pd.DataFrame(topk, columns=['term', 'freq'])

freq_table[['term']] # select term from freq_table

# update freq_table 
# set term = convert_tuple_to_string(term)
# in this case the convert_tuple_to_string function is implemented as a lambda function
# the apply function executes the function on every tuple

freq_table['term'] = freq_table['term'].apply(lambda x: ' '.join(x) if isinstance(x,tuple) else x)

freq_table


***USING COUNTVECTORIZER, TFIDFVECTORIZER***

In [ ]:
#pick up just the Business Section, i.e., Item 1

business = ldf.loc[(ldf['Line'] >= 153) & (ldf['Line'] <= 305), ['Content']] # equivalent to 
                                                 # SELECT Content FROM LDF
                                                 # WHERE Line >= 153 AND Line <= 305
bus_content = pd.DataFrame({'text': ['. '.join(business['Content'].str.strip().tolist())]})

bus_content


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# CountVectorizer does the following:

# lowercases your text (set lowercase=false if you don’t want lowercasing)
# uses utf-8 encoding
# performs tokenization (converts raw text to smaller units of text)
# uses word level tokenization (meaning each word is treated as a separate token)
# ignores single characters during tokenization
# most importantly, it computes a term-document matrix from the corpus

cv = CountVectorizer()
bow = cv.fit_transform(bus_content['text'])

term_list = cv.get_feature_names()
count_list = bow.toarray().sum(axis=0)

print(dict(zip(term_list,count_list)))

In [ ]:
model = CountVectorizer(ngram_range = (1, 3), stop_words='english', )
cv = model.fit_transform(bus_content['text'])
term_list = model.get_feature_names()
count_list = cv.toarray().sum(axis=0)

freq = dict(zip(term_list,count_list))
sorted_freq = sorted(freq.items(), key=lambda x:x[1], reverse=True)
print(sorted_freq)
#fdf = pd.DataFrame.from_dict(sorted_freq)
#print(fdf)

In [ ]:
# A quick refresher for groupby

test = ldf.loc[(ldf['Line'] >= 400) & (ldf['Line'] <= 420), ['Line', 'Sent', 'Content']] 

In [ ]:
# we group by line and concatenate the strings in the 'Content' column

new = pd.DataFrame(test.groupby(['Line'])['Content'].transform(lambda x: '. '.join(x))).drop_duplicates()
new

In [ ]:
# Recover the paragraphs from the ldf dataframe

new = pd.DataFrame(ldf.groupby(['Line'])['Content'].transform(lambda x: '. '.join(x))).drop_duplicates()
new[550:555]

In [ ]:
# First we will apply a stemmer

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
new['stemmed'] = new.Content.map(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))
new.stemmed[550:555]

In [ ]:
# look at https://docs.python.org/3/library/itertools.html
# islice('ABCDEFG', 2, None) --> C D E F G

from itertools import islice
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer


# “min_df” is mimimum document frequency. 
# It sets the minimum number of documents that any term is contained in. 
# It can either be an integer which sets the number specifically, or a decimal between 0 and 1 which is interpreted as a percentage of all documents. 
# “max_df” controls the maximum number of documents any term can be found in. 
# If 90% of documents contain the word “team” then it’s so common that it’s not very useful. 
# “ngram_range” was used before



cvec = CountVectorizer(stop_words='english', min_df=2, max_df=.5, ngram_range=(2,4))

cvec.fit(new.stemmed)
print(len(cvec.vocabulary_))
list(islice(cvec.vocabulary_.items(), 50))

#Let’s look at the top 50 most common terms

cvec_counts = cvec.transform(new.stemmed)

print('sparse matrix shape:', cvec_counts.shape)
print('nonzero count:', cvec_counts.nnz)
print('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))

occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
names = cvec.get_feature_names()
counts_df = pd.DataFrame({'term': names, 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(50)


In [ ]:
cooccurrence = (cvec_counts.T * cvec_counts) # matrix manipulation
cooccurrence.setdiag(0)
cooccurrence_df = pd.DataFrame(data = cooccurrence.toarray(), columns = names, index = names)
cooccurrence_df

In [ ]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

# top 50 terms by average tf-idf weight

weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(50)

In [ ]:
my_stop_words = stopwords.words('english') + ['march', '2020' '2022', '2021', '31', 'end']

tvec = TfidfVectorizer(min_df=.0025, max_df=.1, stop_words=my_stop_words, ngram_range=(2,4))
tvec_weights = tvec.fit_transform(new.stemmed.dropna())
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(50)

In [ ]:
#!pip3 install pytextrank

In [ ]:
#!pip install rake-nltk

In [ ]:
from rake_nltk import Rake

In [ ]:
from nltk.tokenize import WhitespaceTokenizer
r = Rake()

result = []
Tokenizer= WhitespaceTokenizer()

for i, row in new.iterrows():
    if not pd.isna(row['Content']):  #not needed for this example, but it is always good to check in general
        tokens = Tokenizer.tokenize(row['Content'])
        if len(tokens) >= 30:
            t = re.sub(r'[^\w\s]', '', row['Content'])
            r.extract_keywords_from_text(t)
            keywords = set(r.get_ranked_phrases()) 
            result.append([row['Content'], "{" + ", ".join(list(keywords)) + "}"])
        
keywords_df = pd.DataFrame(result)
keywords_df

*** TOWARDS NLP TECHNIQUES***

In [ ]:
# Finding parts of speech for tokens using Spacy

text = 'We are subject to the tax laws in the U.S. at the federal, state, and local government levels and to the tax laws of many other jurisdictions in which we operate or sell products or services. Tax laws might change in ways that adversely affect our tax positions, effective tax rate, and cash flow. The tax laws are extremely complex and subject to varying interpretations. We are subject to tax examinations in various jurisdictions that might assess additional tax liabilities against us. Our tax reporting positions might be challenged by relevant tax authorities, we might incur significant expense in our efforts to defend those challenges, and we might be unsuccessful in those efforts. Developments in examinations and challenges might materially change our provision for taxes in the affected periods and might differ materially from our historical tax accruals. Any of these risks might have a materially adverse impact on our business operations, our cash flows, and our financial position or results of operations'

doc = nlp(text)
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.is_stop)
    

In [ ]:
# Finding noun phrases

# ATTEMPT 1

Nouns = [token.text for token in doc if token.pos_ == "NOUN"]

print(Nouns)

# ATTEMPT 2

chunks = []
for chunk in doc.noun_chunks:
    chunks.append(chunk.text)
print(chunks)

# ATTEMPT 3

pronouns = []

for token in doc:
    if token.pos_ == 'PRON':
        pronouns.append(token.text)
pronouns = list(set(pronouns))
print(pronouns)
chunks = []
for chunk in doc.noun_chunks:
    if chunk.text not in pronouns:
        chunks.append(chunk.text)
print(chunks)    

In [ ]:
# named entities

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

print('--------------')
s = 'FBI Director Christopher Wray said that the agency has assessed that the origin of the COVID-19 pandemic was “most likely a potential lab incident” in Wuhan, China. Wray told Fox News’ Bret Baier in an interview on Tuesday that the FBI’s work on determining where the pandemic originated is continuing, but many details related to the investigation remain classified. '

doc2 = nlp(s)
for ent in doc2.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    